Schema Creation

In [5]:
--Create Schema DDL
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;
GO


IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.007

Dim table 1 - Patients Table

In [6]:
-- Create Patients dim table
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Patients')
BEGIN
    CREATE TABLE dim.Patients(
        PatientKey INT NOT NULL,
        PatientID  [nvarchar](50) NULL,
        PatientName NVARCHAR(50) NOT NULL,
        BirthDate DATE NOT NULL,
        MaritalStatus NVARCHAR(50) NOT NULL,
        Gender NVARCHAR(50) NOT NULL,
        EmailAddress NVARCHAR(50) NOT NULL,
        ServiceKey INT NOT NULL,
        CONSTRAINT PK_Patients PRIMARY KEY NONCLUSTERED (PatientKey ASC)
    );
END;


Commands completed successfully.

Total execution time: 00:00:00.010

Looad data into <span style="color: var(--vscode-foreground);">Dim table 1 - Patients Table</span>

In [7]:

INSERT INTO dim.Patients(PatientKey, PatientID, PatientName, BirthDate, MaritalStatus, Gender, EmailAddress, ServiceKey)
SELECT [PatientKey]
      ,[PatientID]
      ,[PatientName]
      ,[BirthDate]
      ,[MaritalStatus]
      ,[Gender]
      ,[EmailAddress]
      ,[ServiceKey]
FROM [FinalPresent].[dbo].[Patients]
WHERE [FinalPresent].[dbo].[Patients].PatientKey NOT IN (SELECT [PatientKey] FROM [MyDB].dim.Patients)
;
GO

(0 rows affected)

Total execution time: 00:00:00.010

Dim table 2 - Professional Table

In [8]:

-- Create Professional dim table
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Professional')
BEGIN
    CREATE TABLE dim.Professional(
        ProfessionalKey INT NOT NULL,
        ProfessionalID NVARCHAR(50) NOT NULL,
        ProfessionName NVARCHAR(50) NOT NULL,
        Specialisation NVARCHAR(50) NOT NULL,
        Experience INT NOT NULL,
        CONSTRAINT PK_Professional PRIMARY KEY NONCLUSTERED (ProfessionalKey ASC)
    );
END;


Commands completed successfully.

Total execution time: 00:00:00.008

Load Data into <span style="color: var(--vscode-foreground);">Dim table 2 - Professional Table</span>

In [9]:

INSERT INTO dim.Professional(ProfessionalKey, ProfessionalID, ProfessionName, Specialisation, Experience)
SELECT [ProfessionKey]
      ,[ProfessionID]
      ,[ProfessionName]
      ,[Specialisation]
      ,[Experience]
FROM [FinalPresent].[dbo].[Professional]
WHERE [FinalPresent].[dbo].[Professional].ProfessionKey NOT IN (SELECT [ProfessionKey] FROM [MyDB].[dim].[Professional])

;

(0 rows affected)

Total execution time: 00:00:00.006

Dim table 3 - Service Table

In [10]:

-- Create Service dim table
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Service')
BEGIN
    CREATE TABLE dim.Service(
        ServiceKey INT NOT NULL,
        ServiceID  NVARCHAR(50) NOT NULL,
        ServiceName NVARCHAR(100) NOT NULL,
        ServiceDuration INT NOT NULL,
        ServiceCost FLOAT NOT NULL,
        CONSTRAINT PK_Service PRIMARY KEY NONCLUSTERED (ServiceKey ASC)
    );
END;

Commands completed successfully.

Total execution time: 00:00:00.007

Load Data into Table 3 - Service

In [11]:

INSERT INTO dim.Service(ServiceKey, ServiceID, ServiceName, ServiceDuration, ServiceCost)
SELECT [ServiceKey]
      ,[ServiceID]
      ,[ServiceName]
      ,[ServiceDuration]
      ,[ServiceCost]
  FROM [FinalPresent].[dbo].[Service]
WHERE [FinalPresent].[dbo].[Service].ServiceKey NOT IN (SELECT [ServiceKey] FROM [MyDB].[dim].[Service])
;
GO

(0 rows affected)

Total execution time: 00:00:00.006

Fact Table - Appointment Table 4

In [12]:

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'Appointment')
BEGIN
    CREATE TABLE f.Appointment(
        AppointmentKey SMALLINT NOT NULL,
        AppointmentDate NVARCHAR(50) NOT NULL,
        PatientKey INT NOT NULL,
        ProfessionalKey INT NOT NULL,
        ServiceKey INT NULL,
		TotalAmount FLOAT NOT NULL
		CONSTRAINT FK_Appointment_Patients FOREIGN KEY(PatientKey) REFERENCES dim.Patients (PatientKey),
        CONSTRAINT FK_Appointment_Service FOREIGN KEY(ServiceKey) REFERENCES dim.[Service] (ServiceKey),
		CONSTRAINT FK_Appointment_Professional FOREIGN KEY(ProfessionalKey) REFERENCES  dim.Professional (ProfessionalKey)
    );
END;



Commands completed successfully.

Total execution time: 00:00:00.009

Load Data into Fact Table- Appointment

In [13]:

INSERT INTO [MyDB].[f].[Appointment] (AppointmentKey, AppointmentDate, PatientKey, ProfessionalKey, ServiceKey, TotalAmount)
SELECT
    a.[AppointmentKey],
    a.[AppointmentDate],
    a.[PatientKey],
    a.[ProfessionalKey],
    s.[ServiceKey],  
    a.[ServiceCost]
FROM
    [FinalPresent].[dbo].[Appointment] a
JOIN
    [MyDB].[dim].[Service] s ON a.[ServiceKey] = s.[ServiceKey]
WHERE
    a.AppointmentKey NOT IN (SELECT AppointmentKey FROM [MyDB].[f].[Appointment]);

(0 rows affected)

Total execution time: 00:00:00.014